## Run 1

## Install transformers from Huggingface and Simpletransformers



In [3]:
! pip install transformers
! pip install simpletransformers

  Using cached simpletransformers-0.63.4-py3-none-any.whl (248 kB)
  Using cached streamlit-1.5.1-py2.py3-none-any.whl (9.7 MB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached wandb-0.12.10-py2.py3-none-any.whl (1.7 MB)
  Using cached datasets-1.18.3-py3-none-any.whl (311 kB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached yaspin-2.1.0-py3-none-any.whl (18 kB)
  Using cached GitPython-3.1.26-py3-none-any.whl (180 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached shortuuid-1.0.8-py3-none-any.whl (9.5 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached sentry_sdk-1.5.5-py2.py3-none-any.whl (144 kB)
  Using cached xxhash-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl (243 kB)
  Using cached validators-0.18.2-py3-none-any.whl (19 kB)
  Using cached base58-2.1.1-py3-none-any.whl (5.6 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached pydeck-0

## Load Models and Arguments

In [4]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
from google.colab import files

model_args = NERArgs(learning_rate=1e-5, num_train_epochs=4, eval_batch_size=8, train_batch_size=8, weight_decay=1)
model_args.labels_list = ["B-LOC", "I-LOC", "B-ORG", "I-ORG", "B-DATE", "B-PER", "I-PER", "I-DATE", "O"]


model = NERModel(
    "xlmroberta", "xlm-roberta-large", 
    args=model_args,
)

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

## Upload Training txt File


In [5]:
from google.colab import files
uploaded = files.upload()

Saving train_ibo.txt to train_ibo.txt


## Decode and transform into DataFrame for training and testing

In [6]:
def load_train():
  df_text = uploaded['train_ibo.txt'].decode("utf-8")
  text = df_text.split('\n\n')

  df_train = pd.DataFrame()
  df_train['sentence_id'] = pd.Series()
  df_train['words'] = pd.Series()
  df_train['labels'] = pd.Series()

  sentence_num = []
  words = []
  tag = []
  for i in range(len(text)):
      sentence = text[i].split('\n')
      for j in range(len(sentence)-1):
          word = sentence[j].split(" ")
          sentence_num.append(i)
          words.append(word[0])
          tag.append(word[1])
     
  df_train['sentence_id'] = sentence_num
  df_train['words'] = words
  df_train['labels'] = tag
  return df_train

In [7]:
df_train1 = load_train()
df_train1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,sentence_id,words,labels
0,0,Ike,O
1,0,ịda,O
2,0,jụụ,O
3,0,otụ,B-DATE
4,0,nkeji,I-DATE
...,...,...,...
40479,2234,agwa,O
40480,2234,ntutu,O
40481,2234,na,O
40482,2234,-,O


In [8]:
df_train = df_train1.head(5000)
df_test = df_train1.tail(5000)

# Initial model training 

In [11]:
model.train_model(df_train)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/31 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 4:   0%|          | 0/31 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/31 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/31 [00:00<?, ?it/s]

(124, 0.7672873827358407)

# Load and Predict Text

In [9]:
uploaded = files.upload()

Saving ibo_mat_acts.txt to ibo_mat_acts.txt


In [10]:
test_text = uploaded['ibo_mat_acts.txt'].decode("utf-8")
test_text = "".join(filter(lambda x: not x.isdigit(), test_text))
text = test_text.split('\n')
new_text = [s.replace("\x0c", "") for s in text]
#new_text = [s.replace("\x0c", "") for s in text]
new_text = [s.replace("\t", "") for s in new_text]
new_text

['Usoro ọmụmụ Jisọs onye bụ Kraịst ahụ',
 '',
 '  Nke a bụ ndekọ usoro ọmụmụ Jisọs onye bụ Kraịst, nwa Devid, nwa Ebraham.',
 ' ',
 '   Ebraham mụrụ Aịzik,',
 ' Aịzik amụọ Jekọb,',
 ' Jekọb amụọ Juda na ụmụnne ya.',
 '   Juda mụtara Perez na Zera site na Tama,',
 ' Perez mụrụ Hezrọn,',
 ' Hezrọn amụọ Aram.',
 '   Aram mụrụ Aminadab,',
 ' Aminadab mụrụ Nashọn,',
 ' Nashọn amụọ Salmọn.',
 '   Salmọn bụ nna Boaz, site na nwunye ya Rehab,',
 ' Boaz mụrụ Obed, site na nwunye ya Rut,',
 ' Obed amụọ Jesi.',
 '   Jesi amụọ Devid, bụ eze.',
 ' ',
 ' Devid mụrụ Solomọn site na nwunye ya Ụraya.',
 '   Solomọn mụrụ Rehoboam,',
 ' Rehoboam amụọ Abija,',
 ' Abija amụọ Esa.',
 '   Esa mụrụ Jehoshafat,',
 ' Jehoshafat amụọ Joram,',
 ' Joram amụọ Ụzaịa.',
 '   Ụzaịa mụrụ Jotam,',
 ' Jọtam amụọ Ehaz,',
 ' Ehaz amụọ Hezekaya.',
 '   Hezekaya mụrụ Manase,',
 ' Manase amụọ Emọn,',
 ' Emọn amụọ Josaya.',
 '   Mgbe e mere ka ha gaa biri nʼala Babilọn ka Josaya mụrụ Jekonaya na ụmụnne ya.',
 ' ',
 '   Mgbe ha

In [ ]:
#predict_text = uploaded['luo_acts.txt'].decode("utf-8")
#predict_text = predict_text.split('\n')


In [12]:
predictions, raw_outputs = model.predict(new_text)

  0%|          | 0/4 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/239 [00:00<?, ?it/s]

In [ ]:
len(predictions)

1906

# Create new training data

Combine initial training data with the weakly annotated data that was predicted.

In [13]:
def new_data():
  id = len(df_train)

  keys = []
  values = []
  sentence_id = []
  for idx in range(len(predictions)):
    for length in range(len(predictions[idx])):
      for key in predictions[idx][length].keys():
        keys.append(key)
        sentence_id.append(id)
      for value in predictions[idx][length].values():
        values.append(value)

    id += 1 

  df_pred = pd.DataFrame({'sentence_id':sentence_id, 'words':keys, 'labels':values})
  df_newTrain = pd.concat([df_train, df_pred])
  df_newTrain.reset_index(inplace=True, drop=True)
  return df_newTrain

In [14]:
df_newTrain = new_data()
df_newTrain

,sentence_id,words,labels
0,0,Ike,O
1,0,ịda,O
2,0,jụụ,O
3,0,otụ,B-DATE
4,0,nkeji,I-DATE
...,...,...,...
39756,6903,onye,O
39757,6903,ọbụla,O
39758,6903,gbalịrị,O
39759,6903,igbochi,O


# Train 100 Epochs


After each epoch make a prediction and concatenate with training data to create a weak annotated set and repeat 100 times. 

In [15]:
for epoch in range(75):

  print(f"Epoch {epoch+1}")
  model_args = NERArgs(learning_rate=1e-5, num_train_epochs=1, eval_batch_size=8, train_batch_size=8, weight_decay=1, overwrite_output_dir=True)
  model_args.labels_list = ["B-LOC", "I-LOC", "B-ORG", "I-ORG", "B-DATE", "B-PER", "I-PER", "I-DATE", "O"]

  model = NERModel(
      "xlmroberta", "/content/outputs/", 
      args=model_args,
  )

  model.train_model(df_newTrain)
  predictions, raw_outputs = model.predict(new_text)
  df_newTrain = new_data()


Epoch 1


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/204 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/239 [00:00<?, ?it/s]

# Evaluate Model (F1 Score)

In [16]:
result, model_outputs, wrong_preds = model.eval_model(df_test)
result

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'eval_loss': 0.4088232426810547,
 'f1_score': 0.7940841865756542,
 'precision': 0.7653508771929824,
 'recall': 0.8250591016548463}